# Import libraries

In [6]:
import optuna
import os
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
import pandas as pd

In [7]:
def read_file(*args,file_type="csv")->pd.DataFrame:
    """:parameter input directories in sequence
        :return DataFrame"""
    
    path=os.path.join(*args)
    if file_type == "tsv":
        return pd.read_csv(path, sep="\t")
    return pd.read_csv(path)

# Experiments

In [8]:
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

In [9]:
mlflow.set_experiment("DifferentModelsExperiments2")

<Experiment: artifact_location='/home/spynom/mlruns/14', creation_time=1729323387786, experiment_id='14', last_update_time=1729323387786, lifecycle_stage='active', name='DifferentModelsExperiments2', tags={}>

In [10]:
df=read_file("..","data","processed","cleaned.csv").dropna(how="any").drop_duplicates()

X=df.comment
y=df.category+1

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

ngram_range = (1, 3)  # Trigram setting
max_features = 10000  # Set max_features to 1000 for TF-IDF

# Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
vectorizer.fit(X)  # Fit on all vocab data
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)  # Transform test data

In [11]:
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_BalanceWeight_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        train_y_pred=model.predict(X_train)
        test_y_pred = model.predict(X_test)

        # Log accuracy
        train_accuracy = accuracy_score(y_train,train_y_pred)
        test_accuracy = accuracy_score(y_test, test_y_pred)
        
        mlflow.log_metric("train_accuracy", train_accuracy)
        mlflow.log_metric("test_accuracy", test_accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, test_y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

In [12]:
# Step 6: Optuna objective function for XGBoost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42,)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "xgboost"
    log_mlflow("XGBoost", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for XGBoost
run_optuna_experiment()

[I 2024-10-19 13:40:11,564] A new study created in memory with name: no-name-5b107e4d-9b5f-47f6-8df4-90f03147536a
[I 2024-10-19 13:42:23,683] Trial 0 finished with value: 0.6757726269315674 and parameters: {'n_estimators': 93, 'learning_rate': 0.02729171320475009, 'max_depth': 6}. Best is trial 0 with value: 0.6757726269315674.
[I 2024-10-19 13:44:27,778] Trial 1 finished with value: 0.6775662251655629 and parameters: {'n_estimators': 199, 'learning_rate': 0.029752519387702916, 'max_depth': 3}. Best is trial 1 with value: 0.6775662251655629.
[I 2024-10-19 13:49:13,348] Trial 2 finished with value: 0.6531456953642384 and parameters: {'n_estimators': 100, 'learning_rate': 0.0014017443203125704, 'max_depth': 10}. Best is trial 1 with value: 0.6775662251655629.
[I 2024-10-19 13:50:20,101] Trial 3 finished with value: 0.6313465783664459 and parameters: {'n_estimators': 75, 'learning_rate': 0.015259751801683447, 'max_depth': 4}. Best is trial 1 with value: 0.6775662251655629.
[I 2024-10-19 1